In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [2]:
# Load Data
train_data = pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")
original_data = pd.read_csv("/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv")
sample_submission_data = pd.read_csv("/kaggle/input/playground-series-s4e2/sample_submission.csv")


In [3]:
# Merge Data
train_data = train_data.drop("id", axis=1)
train_data = pd.concat([train_data, original_data], ignore_index=True).drop_duplicates()


In [4]:
# Feature Scaling
scaler = StandardScaler()
num_cols = list(train_data.select_dtypes(exclude=['object']).columns)
train_data[num_cols] = scaler.fit_transform(train_data[num_cols])
test_data[num_cols] = scaler.transform(test_data[num_cols])


In [5]:
# Encoding Categorical Variables
labelencoder = LabelEncoder()
object_columns = train_data.select_dtypes(include='object').columns.difference(['NObeyesdad'])

for col_name in object_columns:
    if train_data[col_name].dtypes == 'object':
        train_data[col_name] = labelencoder.fit_transform(train_data[col_name])
        test_data[col_name] = labelencoder.transform(test_data[col_name])


In [6]:
# Define Features and Target
X = train_data.drop(['NObeyesdad'], axis=1)
y = labelencoder.fit_transform(train_data['NObeyesdad'])
X_test = test_data.drop(["id"], axis=1)


In [7]:
# Define LightGBM model parameters
param = {
    "objective": "multiclass",
    "metric": "multi_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "random_state": 42,
    "num_class": 7,
    'learning_rate': 0.030962211546832760,
    'n_estimators': 500,
    'lambda_l1': 0.009667446568254372,
    'lambda_l2': 0.04018641437301800,
    'max_depth': 10,
    'colsample_bytree': 0.40977129346872643,
    'subsample': 0.9535797422450176,
    'min_child_samples': 26
}



In [8]:
# Define Shuffle-Split Cross-Validation
num_splits = 10
ss = ShuffleSplit(n_splits=num_splits, test_size=0.1, random_state=42)

# Initialize lists to store validation accuracies and predictions
val_accuracies = []
test_preds = []

# Perform Shuffle-Split Cross-Validation
for train_index, val_index in ss.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Train LightGBM model
    model_lgb = lgb.LGBMClassifier(**param, verbose=100)
    model_lgb.fit(X_train, y_train)
    
    # Make predictions on validation set
    pred_proba = model_lgb.predict_proba(X_val)
    y_pred = np.argmax(pred_proba, axis=1)
    
    # Calculate accuracy and store
    val_accuracy = accuracy_score(y_val, y_pred)
    val_accuracies.append(val_accuracy)
    
    # Make predictions on test set
    test_pred_proba = model_lgb.predict_proba(X_test)
    test_preds.append(test_pred_proba)



In [9]:
# Average validation accuracy
mean_val_accuracy = np.mean(val_accuracies)
print("Mean Validation Accuracy:", mean_val_accuracy)



Mean Validation Accuracy: 0.915886214442013


In [10]:
# Average predictions on test set
mean_test_preds = np.mean(test_preds, axis=0)
final_test_preds = np.argmax(mean_test_preds, axis=1)



In [11]:
# Decode predictions
pred = labelencoder.inverse_transform(final_test_preds)


In [12]:

# Create submission file
submission = pd.DataFrame({'id': test_data.id, 'NObeyesdad': pred})
submission.to_csv('submission.csv', index=False)
